In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
import string
import json
import spacy 
import time
import pandas as pd
nlp = spacy.load("en_core_web_sm")
from spacy.pipeline.ner import EntityRecognizer
import unicodedata
import numpy as np
from nltk.tokenize import RegexpTokenizer
from collections import Counter
from nltk import FreqDist
import itertools
import re
from nltk.corpus import wordnet

# #Download NLTK resources
# nltk.download('words')
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')

# choose text sample

In [35]:
file=r"C:\Users\marja\Documents\Python_Scripts\Blogspot\scraped_blogs\thewertzone.json"
with open(file, 'r', encoding='utf-8') as file:
        content = json.load(file)
text_values = [item['text'] for item in content[0:50]] #doesnt work for the whole blog
text = ' '.join(text_values)


# normalize unicode, remove digits and put a whitespace before and after ()

In [36]:

text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
text = re.sub(r'\d', '', text)
text=re.sub(r'([()])', r' \1 ', text) #maybe also [] ?

# NLP

In [37]:
document = nlp(text)

# NER and removing some of them from the text

In [38]:
entities = [(ent.text, ent.label_) for ent in document.ents]
entities

[('POSTAfter', 'NORP'),
 ('Patreon', 'PRODUCT'),
 ('Patreon', 'FAC'),
 ('six years', 'DATE'),
 ('half', 'CARDINAL'),
 ('Swiss', 'NORP'),
 ('more than  million', 'CARDINAL'),
 ('Greenland', 'GPE'),
 ('many years', 'DATE'),
 ('a few years old', 'DATE'),
 ('twenty years old', 'DATE'),
 ('Romantasy', 'WORK_OF_ART'),
 ('Sarah J. Maas', 'PERSON'),
 ('Brandon Sanderson', 'PERSON'),
 ('Rebecca Yarros', 'PERSON'),
 ('million', 'CARDINAL'),
 ('a year', 'DATE'),
 ('Patrick Rothfuss', 'PERSON'),
 ('YA', 'ORG'),
 ('Percy Jackson', 'PERSON'),
 ('Joe Abercrombie', 'PERSON'),
 ('Mark Lawrence', 'PERSON'),
 ('Michael J. Sullivan', 'PERSON'),
 ('James Islington', 'PERSON'),
 ('Brandon Sanderson', 'PERSON'),
 ('Robert Jordan', 'PERSON'),
 ('seventeen years ago', 'DATE'),
 ('The Wheel of Time', 'ORG'),
 ('Gospel', 'PERSON'),
 ('JK Rowling', 'PERSON'),
 ('Harry Potter', 'PERSON'),
 ('seriesseventeen years ago', 'DATE'),
 ('Eiichiro Oda', 'PERSON'),
 ('Japanese', 'NORP'),
 ('OnePiece', 'ORG'),
 ('million', 

In [39]:
text_without_ne = [token.text for token in document if token.ent_type_ not in ['GPE', 'PERSON', 'ORDINAL', 'ORG', 'CARDINAL', 'LOC', 'FAC', 'NORP', 'EVENT', 'LAW', 'WORK_OF_ART', 'NER', 'PRODUCT']]
text=(" ".join(text_without_ne))
text

'STICKIED much debate   ( and some requests )   I have signed up with crowdfunding service to better support future blogging efforts . You can find my page here and more information after the jump . After a lengthy break   ( six years since the last version ) , the   ( non- )   patented , utterly non - definitive Wertzone Official SFF All - Time Bestseller List returns . There have been some changes this time . The last list was gettingon for   entries strong , and unreliable and variable reporting meant thelower of the list had more holes in it than cheese after beingvisited by lactose - loving moths , due to patchy reporting . I have limited the numbered list to authorswith copies sold for the sake of sanity . I have left in theremainder of the list from last time , but take those positions and sales figures with a pinchof salt the size of . The usual string of caveats : reporting of sales for authorsis bizarrely spotty , with some authors happy to broadcast their sales , some guardi

# stopwords, punctuation and tokenizing

In [40]:
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(text)


In [41]:
filtered_tokens = [token.text for token in nlp(" ".join(tokens)) if not token.is_stop]
filtered_tokens= [token for token in filtered_tokens if len(token) > 1] #one charactet tokens

# 4. Part-of-speech tagging

In [25]:
pos_tags = pos_tag(filtered_tokens)
pos_tags


[('STICKIED', 'NNP'),
 ('debate', 'NN'),
 ('requests', 'NNS'),
 ('signed', 'VBD'),
 ('crowdfunding', 'VBG'),
 ('service', 'NN'),
 ('better', 'RBR'),
 ('support', 'NN'),
 ('future', 'JJ'),
 ('blogging', 'NN'),
 ('efforts', 'NNS'),
 ('find', 'VBP'),
 ('page', 'JJ'),
 ('information', 'NN'),
 ('jump', 'NN'),
 ('lengthy', 'JJ'),
 ('break', 'NN'),
 ('years', 'NNS'),
 ('version', 'NN'),
 ('non', 'RB'),
 ('patented', 'VBD'),
 ('utterly', 'JJ'),
 ('non', 'RB'),
 ('definitive', 'JJ'),
 ('Wertzone', 'NNP'),
 ('Official', 'NNP'),
 ('SFF', 'NNP'),
 ('Time', 'NNP'),
 ('Bestseller', 'NNP'),
 ('List', 'NNP'),
 ('returns', 'NNS'),
 ('changes', 'NNS'),
 ('time', 'NN'),
 ('list', 'NN'),
 ('gettingon', 'NN'),
 ('entries', 'NNS'),
 ('strong', 'JJ'),
 ('unreliable', 'JJ'),
 ('variable', 'JJ'),
 ('reporting', 'NN'),
 ('meant', 'NN'),
 ('thelower', 'NN'),
 ('list', 'NN'),
 ('holes', 'VBZ'),
 ('cheese', 'JJ'),
 ('beingvisited', 'VBD'),
 ('lactose', 'JJ'),
 ('loving', 'JJ'),
 ('moths', 'NNS'),
 ('patchy', 'IN')

# lemmatize

In [28]:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if no specific POS tag is found

def lemmatize_pos_tokens(pos_tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = []

    for token, pos_tag in pos_tokens:
        pos = get_wordnet_pos(pos_tag)
        lemmatized_token = lemmatizer.lemmatize(token, pos=pos)
        lemmatized_tokens.append(lemmatized_token)

    return lemmatized_tokens

In [30]:
lemmatized_tokens = lemmatize_pos_tokens(pos_tags)
lemmatized_tokens = [token.lower() for token in lemmatized_tokens]